# Argumentative Causal Discovery

Notebook collecting results for causal discovery algorithm ABAPC presented in the Argumentative Causal Discovery paper (Russo, Rapberger and Toni (2024))

In [2]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0,'../../')
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 2000)
sys.path.append('../../ArgCausalDisco/utils/')
from ArgCausalDisco.utils.plotting import *
print(sys.path)

save_figs = True
debug = False
datasets = ['cancer', 'earthquake', 'survey', 'asia']
dags_nodes_map = {'asia':8, 'cancer':5, 'earthquake':5, 'sachs':11, 'survey':6, 'alarm':37, 'child':20, 'insurance':27, 'hailfinder':56, 'hepar2':70}
dags_arcs_map = {'asia':8, 'cancer':4, 'earthquake':4, 'sachs':17, 'survey':6, 'alarm':46, 'child':25, 'insurance':52, 'hailfinder':66, 'hepar2':123}
methods = ['Random', 'FGS', 'NOTEARS-MLP', 'MPC', 'ABAPC (Existing)', 'ABAPC (New)']
names_dict = {'fgs':'FGS', 'nt':'NOTEARS-MLP', 'mpc':'MPC', 'random':'Random', 'abapc':'ABAPC (Existing)', 'ABAPC (New)':'ABAPC (New)'}
symbols_dict = {'abapc':'triangle-down-dot','fgs':'triangle-up-dot','nt':'pentagon-dot','mpc':'hexagon2-dot', 'random':'x'}  
colors_dict = {'abapc':sec_blue,'fgs':sec_orange,'nt':main_purple,'mpc':main_green,'random':'grey', 'ABAPC (New)':'black'}
version = 'bnlearn_50rep' ## for 5000 samples
# version = 'bnlearn_dag_v5_2000' ## for 2000 samples

['../../', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/homes/yh2024/gradual-sem-causal-aba/.venv/lib/python3.12/site-packages', '../../ArgCausalDisco/utils/']


## Main Paper Results

### CPDAG Evaluation

In [6]:
version_cpdag = version+'_cpdag'
all_sum = pd.read_csv(f"../../results_pure_aba/stored_results_{version}_cpdag.csv")
all_sum

,dataset,model,elapsed_mean,elapsed_std,nnz_mean,nnz_std,fdr_mean,fdr_std,tpr_mean,tpr_std,...,recall_mean,recall_std,F1_mean,F1_std,shd_mean,shd_std,sid_low_mean,sid_low_std,sid_high_mean,sid_high_std
0,cancer,ABAPC (ASPforABA),0.47,0.24,3.86,0.64,0.53,0.11,0.46,0.13,...,0.44,0.15,0.47,0.11,2.34,0.69,9.74,1.48,10.64,1.54
1,cancer,Random,0.00,0.00,4.00,0.00,0.70,0.24,0.30,0.24,...,0.20,0.19,0.32,0.13,5.72,1.55,7.94,3.63,14.04,3.03
2,cancer,MPC,0.00,0.00,3.76,0.52,0.01,0.04,0.93,0.11,...,0.50,0.00,0.52,0.04,2.04,0.20,3.88,1.81,13.88,1.81
3,cancer,ABAPC (Ours),0.96,1.00,3.86,0.64,0.53,0.11,0.46,0.13,...,0.44,0.15,0.47,0.11,2.34,0.69,9.74,1.48,10.64,1.54
4,cancer,NOTEARS-MLP,5.69,3.72,1.92,0.49,0.43,0.20,0.25,0.00,...,0.25,0.00,0.34,0.03,3.00,0.00,9.44,1.54,10.24,0.43
5,earthquake,ABAPC (ASPforABA),0.27,0.33,4.60,0.67,0.11,0.12,1.00,0.00,...,1.00,0.00,0.94,0.07,0.60,0.67,0.00,0.00,9.28,4.51
6,earthquake,Random,0.00,0.00,4.00,0.00,0.72,0.21,0.28,0.21,...,0.16,0.17,0.30,0.10,5.80,1.48,7.88,2.88,13.88,3.46
7,earthquake,MPC,0.00,0.00,4.64,0.56,0.13,0.11,1.00,0.00,...,0.50,0.00,0.46,0.03,2.64,0.56,5.00,0.00,15.00,0.00
8,earthquake,ABAPC (Ours),0.58,0.68,4.58,0.61,0.11,0.11,1.00,0.00,...,1.00,0.00,0.94,0.06,0.58,0.61,0.00,0.00,9.40,4.66
9,earthquake,NOTEARS-MLP,41.55,8.63,5.24,0.66,0.33,0.16,0.86,0.18,...,0.48,0.08,0.41,0.08,3.38,0.78,2.28,2.95,14.60,1.56


In [7]:
version_cpdag = version+'_cpdag'
all_sum = pd.read_csv(f"../../results_pure_aba/stored_results_{version}_cpdag.csv")
all_sum['model'][all_sum['model']=='ABAPC (Ours)'] = 'ABAPC (Existing)'
all_sum['model'][all_sum['model']=='ABAPC (ASPforABA)'] = 'ABAPC (New)'


all_sum['n_edges'] = all_sum['dataset'].map(dags_arcs_map)
all_sum['n_nodes'] = all_sum['dataset'].map(dags_nodes_map)
for var in ['shd','SID_low','SID_high']:
    all_sum['p_'+var+'_mean'] = all_sum[var.lower()+'_mean'].astype(float)/all_sum['n_edges'].astype(int)
    all_sum['p_'+var+'_std'] = all_sum[var.lower()+'_std'].astype(float)/all_sum['n_edges'].astype(int)
all_sum['dataset'] = [a.upper() for a in all_sum["dataset"].astype(str)]
all_sum['dataset'] = all_sum['dataset'] +np.repeat("<br> |V|=",len(all_sum))+ all_sum["n_nodes"].astype(str)+np.repeat(", |E|=",len(all_sum))+\
                     all_sum["n_edges"].astype(str)

double_bar_chart_plotly(all_sum, ['p_SID_low','p_SID_high'], names_dict, colors_dict, methods, save_figs=save_figs, output_name="./Fig.2_SID_cpdag.html", debug=False, range_y1=[0,6], range_y2=[0,6])#